# このノートブックの概要

- Santa 2022
- 各象限に対して or-tools による整数最適化。

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/santa-2022/sample_submission.csv
/kaggle/input/santa-2022/image.png
/kaggle/input/santa-2022/image.csv


In [2]:
import matplotlib.collections as mc
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import io
import pandas as pd
import pickle
import random
import time
from functools import *
from itertools import *
from pathlib import Path
from PIL import Image

In [3]:
data_dir = Path('/kaggle/input/santa-2022')
DEBUG = False

In [4]:
# ファイル読み込み

def df_to_image(df):
    side = int(len(df) ** 0.5)  # assumes a square image
    im = df.set_index(['x', 'y']).to_numpy().reshape(side, side, -1)
    # Flip X axis and transpose X-Y axes to simplify cartesian to array mapping:
    im = im[::-1,:,:]
    im = np.transpose(im, (1, 0, 2))
    return im

df_image = pd.read_csv(data_dir / 'image.csv')
image = df_to_image(df_image)

In [5]:
# 格子点の座標を config の腕の長さに合わせて値を決める関数。

def _arms_value(x, arm_len):
    
    assert arm_len > 0

    if x > 0 and x > arm_len:
        _x = arm_len
    elif x > 0 and x <= arm_len:
        _x = x
    elif x == 0:
        _x = 0
    elif x < 0 and x < -arm_len:
        _x = -arm_len
    elif x < 0 and x >= -arm_len:
        _x = x

    return _x


assert _arms_value(0, 2) == 0
assert _arms_value(1, 2) == 1
assert _arms_value(2, 2) == 2
assert _arms_value(5, 2) == 2
assert _arms_value(-1, 2) == -1
assert _arms_value(-2, 2) == -2
assert _arms_value(-3, 2) == -2

assert _arms_value(0, 32) == 0
assert _arms_value(1, 32) == 1
assert _arms_value(32, 32) == 32
assert _arms_value(33, 32) == 32
assert _arms_value(-1, 32) == -1
assert _arms_value(-32, 32) == -32
assert _arms_value(-33, 32) == -32


# 格子点の座標を config から一意な config を得る関数。
def _get_arm_values(x, fixed, variable):
    x -= sum(fixed)
    tmp = []
    for i in reversed(variable):
        _x = _arms_value(x, i)
        x -= _x
        tmp.append(_x)
    
    if len(fixed) == 0:
        tmp.reverse()
        fixed = tmp
    elif abs(fixed[0]) > abs(variable[0]):
        tmp.reverse()
        fixed.extend(tmp)
    else:
        fixed = tmp + fixed
    return fixed


assert _get_arm_values(1, [64], [32, 16, 8, 4, 2, 1, 1])        == [64, -31, -16, -8, -4, -2, -1, -1]
assert _get_arm_values(1, [32, 16, 8, 4, 2, 1, 1], [64])        == [-63, 32, 16, 8, 4, 2, 1, 1]
assert _get_arm_values(-1, [-64], [32, 16, 8, 4, 2, 1, 1])      == [-64, 31, 16, 8, 4, 2, 1, 1]
assert _get_arm_values(0, [-32, -16, -8, -4, -2, -1, -1], [64]) == [64, -32, -16, -8, -4, -2, -1, -1]


def _upper_right(x, y):
    
    fixed_x = [64]
    fixed_y = [32, 16, 8, 4, 2, 1, 1]
    variable_x = [32, 16, 8, 4, 2, 1, 1]
    variable_y = [64]
    
    fixed_x = _get_arm_values(x, fixed_x, variable_x)
    fixed_y = _get_arm_values(y, fixed_y, variable_y)
    
    return [(_x, _y) for _x, _y in zip(fixed_x, fixed_y)]


assert (0, 0) == tuple(np.array(_upper_right(0, 0)).sum(axis=0))
assert (0, 1) == tuple(np.array(_upper_right(0, 1)).sum(axis=0))
assert (1, 0) == tuple(np.array(_upper_right(1, 0)).sum(axis=0))
assert (128, 128) == tuple(np.array(_upper_right(128, 128)).sum(axis=0))


def _upper_left(x, y):
    
    fixed_x = [-32, -16, -8, -4, -2, -1, -1]
    fixed_y = [64]
    variable_x = [64]
    variable_y = [32, 16, 8, 4, 2, 1, 1]
    
    fixed_x = _get_arm_values(x, fixed_x, variable_x)
    fixed_y = _get_arm_values(y, fixed_y, variable_y)
    
    return [(_x, _y) for _x, _y in zip(fixed_x, fixed_y)]


assert (-1, 0) == tuple(np.array(_upper_left(-1, 0)).sum(axis=0))
assert (0, 1) == tuple(np.array(_upper_left(0, 1)).sum(axis=0))
assert (-1, 1) == tuple(np.array(_upper_left(-1, 1)).sum(axis=0))
assert (0, 128) == tuple(np.array(_upper_left(0, 128)).sum(axis=0))
assert (-128, 0) == tuple(np.array(_upper_left(-128, 0)).sum(axis=0))
assert (-128, 128) == tuple(np.array(_upper_left(-128, 128)).sum(axis=0))


def _under_left(x, y):
        
    fixed_x = [-64]
    fixed_y = [-32, -16, -8, -4, -2, -1, -1]
    variable_x = [32, 16, 8, 4, 2, 1, 1]
    variable_y = [64]
    
    fixed_x = _get_arm_values(x, fixed_x, variable_x)
    fixed_y = _get_arm_values(y, fixed_y, variable_y)
    
    return [(_x, _y) for _x, _y in zip(fixed_x, fixed_y)]


assert (-1, 0) == tuple(np.array(_under_left(-1, 0)).sum(axis=0))
assert (0, -1) == tuple(np.array(_under_left(0, -1)).sum(axis=0))
assert (-1, -1) == tuple(np.array(_under_left(-1, -1)).sum(axis=0))
assert (0, -128) == tuple(np.array(_under_left(0, -128)).sum(axis=0))
assert (-128, 0) == tuple(np.array(_under_left(-128, 0)).sum(axis=0))
assert (-128, -128) == tuple(np.array(_under_left(-128, -128)).sum(axis=0))


def _under_right(x, y):
    
    fixed_x = [32, 16, 8, 4, 2, 1, 1]
    fixed_y = [-64]
    variable_x = [64]
    variable_y = [32, 16, 8, 4, 2, 1, 1]
    
    fixed_x = _get_arm_values(x, fixed_x, variable_x)
    fixed_y = _get_arm_values(y, fixed_y, variable_y)
    
    return [(_x, _y) for _x, _y in zip(fixed_x, fixed_y)]


assert (1, 0) == tuple(np.array(_under_right(1, 0)).sum(axis=0))
assert (1, -1) == tuple(np.array(_under_right(1, -1)).sum(axis=0))
assert (0, -1) == tuple(np.array(_under_right(0, -1)).sum(axis=0))
assert (128, 0) == tuple(np.array(_under_right(128, 0)).sum(axis=0))
assert (0, -128) == tuple(np.array(_under_right(0, -128)).sum(axis=0))
assert (128, -128) == tuple(np.array(_under_right(128, -128)).sum(axis=0))

assert _upper_left(0, 128) == _upper_right(0, 128)
assert _upper_right(128, 0) == _under_right(128, 0)
assert _under_right(0, -128) == _under_left(0, -128)
assert _upper_left(-128, 0) == _under_left(-128, 0)


# y 軸上の格子点から config を返す関数。
# 原点の config が指定されていることへの対応。
def _on_axis_y(x, y):
    fixed_x = [64, -32, -16, -8, -4, -2, -1, -1]
    fixed_y = []
    variable_x = []
    variable_y = [64, 32, 16, 8, 4, 2, 1, 1]
    
    fixed_y = _get_arm_values(y, fixed_y, variable_y)
    
    return [(_x, _y) for _x, _y in zip(fixed_x, fixed_y)]

In [6]:
# 初期解となる格子点系列の生成する関数。
def get_initial_points():
    
    n = 128
    points = []
        
    # 第1象限
    for y in range(0, n+1):
        points.append((0, y))
    for y in range(n, -1, -1):
        if y % 2 == 0:
            for x in range(1, n+1):
                points.append((x, y))
        else:
            for x in range(n, 0, -1):
                points.append((x, y))
    
    # 第4象限
    for y in range(-1, -n-1, -1):
        points.append((128, y))
    for y in range(-n, -2):
        if y % 2 == 0:
            for x in range(n-1, 0, -1):
                points.append((x, y))
        else:
            for x in range(1, n):
                points.append((x, y))
    for x in range(n-1, 0, -1):
        if x % 2 == 0:
            for y in range(-1, -3, -1):
                points.append((x, y))
        else:
            for y in range(-2, 0):
                points.append((x, y))
                
                
    # 第3象限
    for y in range(-1, -n-1, -1):
        points.append((0, y))

    for y in range(-n, 1):
        if y % 2 == 0:
            for x in range(-1, -n-1, -1):
                points.append((x, y))
        else:
            for x in range(-n, 0):
                points.append((x, y))
                
    # 第2象限
    for y in range(1, n-1):
        if y % 2 == 0:
            for x in range(-1, -n-1, -1):
                points.append((x, y))
        else:
            for x in range(-n, 0):
                points.append((x, y))
    for x in range(-n, 0):
        if x % 2 == 0:
            for y in range(n-1, n+1):
                points.append((x, y))
        else:
            for y in range(n, n-2, -1):
                points.append((x, y))
    for y in range(n, -1, -1):
        points.append((0, y))

    return points


# 格子点を一意な config に変換する関数。
def to_config(point):
    # 一意に決まらないのでルール化する。
    # 具体的には下記のように表す。
    # 第1象限が [(64, x), (x, 32), (x, 16), (x, 8), (x, 4), (x, 2), (x, 1), (x, 1)] で表す。
    # 第2象限が [(x, 64), (-32, x), (-16, x), (-8, x), (-4, x), (-2, x), (-1, x), (-1, x)] で表す。
    # 第3象限が [(-64, x), (x, -32), (x, -16), (x, -8), (x, -4), (x, -2), (x, -1), (x, -1)] で表す。
    # 第4象限が [(x, -64), (32, x), (16, x), (8, x), (4, x), (2, x), (1, x), (1, x)] で表す。
    # 
    # 原点近くは短い腕から動かす。
    # 原点 [0, 0] は [(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)]
    # このとき [1, 1] は [(64, 0), (0, 32), (0, 16), (0, 8), (0, 4), (0, 2), (1, 1), (1, 1)] とする。
    # ただし [0, 0] は制限があるため、{[0, y]: 0 <= y <= 128} は特別なルールを設ける。
    
    x, y = point
    if x == 0 and 0 <= y:
        _config = _on_axis_y(x, y)
    elif 0 < x and 0 <= y:
        _config = _upper_right(x, y)
    elif x < 0 and 0 < y:
        _config = _upper_left(x, y)
    elif x < 0 and y <= 0:
        _config = _under_left(x, y)
    elif 0 <= x and y < 0:
        _config = _under_right(x, y)
    
    return _config


# 格子点系列を一意な config 系列に変換する関数。
def to_configs(points):
    return [to_config(point) for point in points]


def points_cost_index(points, image):
    p_cost = [step_cost(p, q, image) for p, q in zip(points[:-1], points[1:])]
    p_cost_index = [i[0] for i in sorted(enumerate(p_cost), key=lambda x: x[1])]
    p_cost_index.reverse()
    return p_cost_index

def step_cost(p, q, image):
    ix, iy = p
    jx, jy = q
    ix += 128
    iy += 128
    jx += 128
    jy += 128
    d = np.abs(ix-jx)+np.abs(iy-jy)
    return np.sum(np.abs(image[ix,iy]-image[jx,jy]))*3 + np.sqrt(d)

In [7]:
def in_bounding_box(point):
    if -128 <= point[0] and point[0] <= 128 and -128 <= point[1] and point[1] <= 128:
        return point
    else:
        return None


def neighbor(p):
    
    x, y = p
    
    if x == 0 and 0 <= y:
        nei = []
        # if y == 0:
        #     nei = [
        #         ( p[0]  , p[1]+1 )
        #     ]
        # elif y == 128:
        #     nei = [
        #         ( p[0]  , p[1]-1 ),
        #         # ( p[0]+1, p[1]   ),
        #     ]
        # else:
        #     nei = [
        #         ( p[0]  , p[1]+1 ),
        #         ( p[0]  , p[1]-1 ),
        #     ]
    elif 0 < x and 0 <= y:
        if x == 1 and y == 0:
            nei = [
                ( p[0]+1, p[1]   ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]+1 )
            ]
        elif x == 1 and y == 128:
            nei = [
                # ( p[0]-1, p[1]   ),
                ( p[0]+1, p[1]   ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1)
            ]
        elif x == 128 and y == 0:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 ),
                # ( p[0]  , p[1]-1 )
            ]
        elif x == 128 and y == 128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]  , p[1]-1 ),
                ( p[0]-1, p[1]-1 )
            ]
        elif x == 1:
            nei = [
                ( p[0]  , p[1]-1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 ),
                ( p[0]+1, p[1]+1 )
            ]
        elif x == 128:
            nei = [
                ( p[0]  , p[1]-1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]-1, p[1]+1 )
            ]
        elif y == 0:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        elif y == 128:
            nei = [
                ( p[0]-1, p[1]-1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        else:
            nei =  [
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1 ),
            ]
    elif 0 <= x and y < 0:
        if x == 0 and y == -1:
            nei = [
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif x == 0 and y == -128:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        elif x == 128 and y == -1:
            nei = [
                # ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]  , p[1]-1 ),
                ( p[0]-1, p[1]-1 )
            ]
        elif x == 128 and y == -128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 )
            ]
        elif x == 0:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif x == 128:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 )
            ]
        elif y == -1:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif y == -128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        else:
            nei =  [
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1 ),
            ]
    elif x < 0 and y <= 0:
        if x == -1 and y == 0:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 )
            ]
        elif x == -1 and y == -128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 )
            ]
        elif x == -128 and y == 0:
            nei = [
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif x == -128 and y == -128:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        elif x == -1:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 )
            ]
        elif x == -128:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif y == 0:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif y == -128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        else:
            nei =  [
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1 )
            ]
    elif x < 0 and 0 < y:
        if x == -1 and y == 1:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 )
            ]
        elif x == -1 and y == 128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 )
            ]
        elif x == -128 and y == 1:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 )
            ]
        elif x == -128 and y == 128:
            nei = [
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif x == -1:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 )
            ]
        elif x == -128:
            nei = [
                ( p[0]  , p[1]+1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]-1 )
            ]
        elif y == 1:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]+1, p[1]+1 )
            ]
        elif y == 128:
            nei = [
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1 ),
                ( p[0]+1, p[1]-1 )
            ]
        else:
            nei =  [
                ( p[0]+1, p[1]   ),
                ( p[0]+1, p[1]+1 ),
                ( p[0]  , p[1]+1 ),
                ( p[0]-1, p[1]+1 ),
                ( p[0]-1, p[1]   ),
                ( p[0]-1, p[1]-1 ),
                ( p[0]  , p[1]-1 ),
                ( p[0]+1, p[1]-1 ),
            ]
    
    return [p for p in nei if in_bounding_box(p)]


assert set(neighbor((0, 0))) == set([])
assert set(neighbor((0, 1))) == set([])
assert set(neighbor((0, 128))) == set([])
assert set(neighbor((1, 128))) == set([(1, 127), (2, 128), (2, 127)])
assert set(neighbor((1, 1))) == set([(1, 2), (1, 0), (2, 2), (2, 1), (2, 0)])
assert set(neighbor((1, 0))) == set([(1, 1), (2, 1), (2, 0)])
assert set(neighbor((128, 0))) == set([(127, 0), (127, 1), (128, 1)])
assert set(neighbor((128, 127))) == set([(127, 128), (127, 127), (127, 126), (128, 128), (128, 126)])
assert set(neighbor((128, 128))) == set([(127, 128), (127, 127), (128, 127)])

assert set(neighbor((0, -1))) == set([(0, -2), (1, -1), (1, -2)])
assert set(neighbor((0, -128))) == set([(0, -127), (1, -127), (1, -128)])
assert set(neighbor((1, -1))) == set([(0, -1), (0, -2), (1, -2), (2, -1), (2, -2)])
assert set(neighbor((1, -128))) == set([(0, -128), (0, -127), (1, -127), (2, -127), (2, -128)])
assert set(neighbor((128, -1))) == set([(127, -1), (127, -2), (128, -2)])
assert set(neighbor((128, -2))) == set([(127, -1), (127, -2), (127, -3), (128, -1), (128, -3)])
assert set(neighbor((128, -128))) == set([(127, -128), (127, -127), (128, -127)])

assert set(neighbor((-1, 0))) == set([(-2, 0), (-2, -1), (-1, -1)])
assert set(neighbor((-1, -128))) == set([(-2, -128), (-2, -127), (-1, -127)])
assert set(neighbor((-128, 0))) == set([(-128, -1), (-127, 0), (-127, -1)])
assert set(neighbor((-128, -128))) == set([(-128, -127), (-127, -128), (-127, -127)])
assert set(neighbor((-2, 0))) == set([(-3, 0), (-3, -1), (-2, -1), (-1, 0), (-1, -1)])
assert set(neighbor((-128, -127))) == set([(-128, -126), (-128, -128), (-127, -126), (-127, -127), (-127, -128)])

assert set(neighbor((-1, 1))) == set([(-1, 2), (-2, 1), (-2, 2)])
assert set(neighbor((-1, 128))) == set([(-1, 127), (-2, 128), (-2, 127)])
assert set(neighbor((-128, 1))) == set([(-128, 2), (-127, 1), (-127, 2)])
assert set(neighbor((-128, 128))) == set([(-128, 127), (-127, 128), (-127, 127)])
assert set(neighbor((-1, 2))) == set([(-1, 3), (-1, 1), (-2, 3), (-2, 2), (-2, 1)])
assert set(neighbor((-128, 2))) == set([(-128, 3), (-128, 1), (-127, 3), (-127, 2), (-127, 1)])

In [8]:
from ortools.linear_solver import pywraplp

def get_solver(nodes, s, t, c, hashed_neighbors, subtours=[]):

    solver = pywraplp.Solver('tsp', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # x = [[solver.IntVar(0, 1, f'') for i in range(n)] for j in range(n)]
    x = {(i, j): solver.IntVar(0, 1, f'') for i, nei in hashed_neighbors.items() for j in nei if i in nodes and j in nodes}
    
    # for i in range(n):
    for i, nei in hashed_neighbors.items():
        # for j in range(n):
        for j in nei:
            # if i==j:
            if i==j and i in nodes and j in nodes:
                # solver.Add(x[i][j] == 0)
                solver.Add(x[i, j] == 0)
            # else:
            elif i in nodes and j in nodes:
                # solver.Add(x[i][j] + x[j][i] <= 1)
                solver.Add(x[i, j] + x[j, i] <= 1)
    
    # solver.Add(solver.Sum([x[s][j] for j in range(n)]) == 1)
    # solver.Add(solver.Sum([x[i][t] for i in range(n)]) == 1)
    # solver.Add(solver.Sum([x[t][j] for j in range(n)]) == 0)
    # solver.Add(solver.Sum([x[i][s] for i in range(n)]) == 0)
    solver.Add(solver.Sum([x[s, j] for j in hashed_neighbors[s]]) == 1)
    solver.Add(solver.Sum([x[i, t] for i in hashed_neighbors[t]]) == 1)
    solver.Add(solver.Sum([x[t, j] for j in hashed_neighbors[t]]) == 0)
    solver.Add(solver.Sum([x[i, s] for i in hashed_neighbors[s]]) == 0)

    # for j in range(n):
    for j in nodes:
        if j not in [s, t]:
            # solver.Add(solver.Sum([x[i][j] for i in range(n)]) == 1)
            # solver.Add(solver.Sum([x[j][k] for k in range(n)]) == 1)
            solver.Add(solver.Sum([x[i, j] for i in hashed_neighbors[j]]) == 1)
            solver.Add(solver.Sum([x[j, k] for k in hashed_neighbors[j]]) == 1)

    if subtours is not []:
        for subtour in subtours:
            # _subtour = [i for i in range(n) if i not in subtour]
            # solver.Add(solver.Sum([x[i][j] for i in subtour for j in _subtour]) >= 1)
            _subtour = [i for i in nodes if i not in subtour]
            solver.Add(solver.Sum([x[i, j] for i in subtour for j in _subtour]) >= 1)

    obj = solver.Sum([c[i, j] * x[i, j] for i in nodes for j in hashed_neighbors[i]])
    solver.Minimize(obj)
    return solver, x


def get_subours(nodes, s, x):

    # 部分巡回路を検出する。
    subtours = []
    start = s
    while True:
        subtour = [start]
        got_subtour = False
        # for _ in range(n):
        for _ in nodes:
            # for j in range(n):
            for j in nodes:
                if int(x[start, j].SolutionValue()) == 1 and j!=subtour[0]:
                    # print(f'{start} -> {j}')
                    subtour.append(j)
                    start = j
                    break
                elif int(x[start, j].SolutionValue()) == 1 and j==subtour[0]:
                    got_subtour = True
                    break

            if got_subtour:
                break
        
        subtours.append(subtour)

        # 部分巡回路で訪問した都市数を確認する。
        visited = [i for subtour in subtours for i in subtour]

        # すべての都市を訪問したら終了する。
        # そうでなければ未訪問の都市をスタートとして部分巡回路の検出を継続する。
        # if len(visited) == n:
        if len(visited) == len(nodes):
            break
        else:
            # for i in range(n):
            for i in nodes:
                if i not in visited:
                    start = i
                    break

    return subtours


def ip_solve(nodes, s, t, c, hashed_neighbors):
    count = 0
    subtours = []
    while True:
        
        count += 1
        
        # モデル化して求解する。
        solver, x = get_solver(nodes, s, t, c, hashed_neighbors, subtours)
        status = solver.Solve()

        if status:
            raise Exception('status error!')

        # 球解ステータスを確認する。
        objval = solver.Objective().Value()
        _subtours = get_subours(nodes, s, x)
        print(f"Iteration {count}'s optimal value: {objval:.4f}"
            f'  number of subtours: {len(_subtours)}')

        # 巡回路が1つだけなら終了する。
        # そうでないなら、部分巡回路を更新して次のステップへ進む。
        if len(_subtours) == 1:
            tour = _subtours
            break
        else:
            subtours += _subtours
        
    return tour

In [9]:
# 格子点番号を格子点座標に戻す関数。
def hash_to_xy(i):
    return i//257, i%257

# 格子点座標を格子点番号に変換する関数。
def xy_to_hash(x,y):
    return x*257+y

# 格子点を4つの象限にわけてリストに格納する。
tr, br, bl, tl = [], [], [], []
for x in range(-128, 129):
    for y in range(-128, 129):
        
        if x == 0 and 0 <= y:
            pass
        elif 0 < x and 0 <= y:
            tr.append((x, y))
        elif 0 <= x and y < 0:
            br.append((x, y))
        elif x < 0 and y <= 0:
            bl.append((x, y))
        elif x < 0 and 0 < y:
            tl.append((x, y))

# 各象限の格子点番号をを得る。
tr_hash = [xy_to_hash(x, y) for x, y in tr]
br_hash = [xy_to_hash(x, y) for x, y in br]
bl_hash = [xy_to_hash(x, y) for x, y in bl]
tl_hash = [xy_to_hash(x, y) for x, y in tl]

# 格子点番号をもとに近傍を得る。
neighbors = {(x, y): neighbor((x, y)) for x in range(-128, 129) for y in range(-128, 129)}
hashed_neighbors = {xy_to_hash(*p): [xy_to_hash(*q) for q in nei] for p, nei in neighbors.items()}

# 格子点番号の近傍をもとにコストを計算する。
cost_dic = {(xy_to_hash(*p), xy_to_hash(*q)): step_cost(p, q, image) for p, nei in neighbors.items() for q in nei }

In [ ]:
# 各象限に対して、最適化問題を解く。
tr_points = ip_solve(tr_hash, 
                     xy_to_hash(1, 128), 
                     xy_to_hash(128, 0), 
                     cost_dic, 
                     hashed_neighbors)

import pickle

with open('tr_points.pkl', 'wb') as f:
    pickle.dump(tr_points, f)

In [ ]:
br_points = ip_solve(br_hash,
                     xy_to_hash(128, -1),
                     xy_to_hash(0, -128),
                     cost_dic,
                     hashed_neighbors)

with open('br_points.pkl', 'wb') as f:
    pickle.dump(br_points, f)

In [ ]:
bl_points = ip_solve(bl_hash,
                     xy_to_hash(-1, -128),
                     xy_to_hash(-128, 0),
                     cost_dic,
                     hashed_neighbors)

with open('bl_points.pkl', 'wb') as f:
    pickle.dump(bl_points, f)

In [ ]:
tl_points = ip_solve(tl_hash,
                     xy_to_hash(-128, 1),
                     xy_to_hash(-1, 128),
                     cost_dic,
                     hashed_neighbors)

with open('tl_points.pkl', 'wb') as f:
    pickle.dump(tl_points, f)